<a href="https://colab.research.google.com/github/sun9huni/first-repository/blob/main/sunghoon_notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

modu_ds_4_credit_card_fraud_detection_path = kagglehub.competition_download('modu-ds-4-credit-card-fraud-detection')

print('Data source import complete.')


In [ ]:
!pip install -U scikit-learn==1.4.2 imbalanced-learn==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 77.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# 필요 라이브러리 임포트
from sklearn.model_selection import train_test_split, RandomizedSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score, average_precision_score, precision_recall_curve, roc_curve
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.pipeline import Pipeline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

train = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/train.csv")
test = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/test.csv")

train.head()

test.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,170883,120396.0,-2.410892,-1.264333,-0.787460,-0.212152,1.139859,-2.030200,0.533818,-0.194852,...,-0.558831,-0.076699,0.054816,-1.243901,0.147148,0.974330,1.309377,0.083660,-0.611166,124.98
1,170884,120396.0,-1.394526,0.284790,2.979226,0.362345,-1.344546,2.278858,-1.836101,-1.797895,...,-0.807798,1.945416,-0.539926,-0.446763,-0.521142,0.405680,-0.149237,0.458944,-0.071516,50.00
2,170885,120396.0,0.140378,0.989574,-0.506447,0.480553,1.407763,0.371637,0.811775,0.067702,...,0.120841,0.385872,1.302941,-0.177157,-0.192410,-0.724567,-0.433276,0.540947,0.323866,8.69
3,170886,120396.0,-1.303150,0.187865,0.944395,-1.277834,1.236744,-1.301696,1.402234,-0.558914,...,-0.251346,-0.274175,-0.656826,-0.164985,0.001284,-0.001842,-0.024050,-0.436872,0.051705,59.99
4,170887,120396.0,1.947883,-1.095332,-1.392121,-0.600598,-0.320840,0.048953,-0.443486,-0.081011,...,-0.293448,-0.250594,-0.385151,0.099030,0.221922,-0.212846,0.586637,-0.071616,-0.045475,119.97


In [ ]:
# 데이터 탐색 및 시각화 함수

def explore_data(train):
    """
    데이터셋의 기본 정보와 분포, 상관관계 시각화를 수행하여 데이터 이해를 돕는다.
    """
    print("데이터셋 기본 정보:")
    print(f"훈련 데이터 형태: {train.shape}")
    print("\n클래스 분포:")
    print(train["Class"].value_counts())
    print(train["Class"].value_counts(normalize=True) * 100)

    plt.figure(figsize=(10, 8))
    # 거래 금액 분포 시각화
    plt.subplot(2, 2, 1)
    sns.histplot(train["Amount"], bins=50)
    plt.title("Amount Distribution")
    plt.xlabel("Amount")
    plt.xscale('log')

    # 거래 시간 분포
    plt.subplot(2, 2, 2)
    sns.histplot(train["Time"], bins=50)
    plt.title("Time Distribution")
    plt.xlabel("Time(second)")

    # 클래스별 금액 분포
    plt.subplot(2, 2, 3)
    sns.boxplot(x="Class", y="Amount", data=train)
    plt.title("Class Amount Distribution")
    plt.yscale('log')

    # 주요 피처 상관관계 히트맵
    plt.subplot(2, 2, 4)
    corr = train[["V1", "V2", "V3", "V4", "V5", "Amount", "Class"]].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Feature correlation")
    plt.tight_layout()
    plt.show()

In [ ]:
# 데이터 전처리 함수

def get_preprocessed_df(df=None, train_mode=True):
    """
    원본 데이터에서 필요 피처를 선택하고, 로그 변환 및 시간대 피처를 생성한다.
    이상치 제거는 훈련 데이터에서만 수행한다.
    """
    df_copy = df.copy()

    # 거래 금액 로그 변환
    df_copy["Amount_Scaled"] = np.log1p(df_copy["Amount"])

    # 거래 시간으로부터 시간대 피처 생성
    df_copy["Hour"] = np.floor(df_copy["Time"] / 3600) % 24

    # 사용하지 않는 원본 피처 제거
    df_copy.drop(["Time", "Amount"], axis=1, inplace=True)

    # 훈련 데이터인 경우 이상치 제거 수행
    if train_mode and "Class" in df_copy.columns:
        outlier_columns = ["V1", "V2", "V4", "V10", "V14"]
        outlier_indices = set()
        for column in outlier_columns:
            indices = get_outlier(df=df_copy, column=column, weight=1.5)
            outlier_indices.update(indices)
        print(f"제거된 이상치 데이터 수: {len(outlier_indices)}")
        df_copy.drop(list(outlier_indices), axis=0, inplace=True)

    return df_copy

In [ ]:
# 이상치 탐지 함수

def get_outlier(df=None, column=None, weight=1.5):
    """
    정상 거래 데이터에서 IQR 기반 이상치를 탐지하여 인덱스 리스트 반환.
    정상 거래만 고려하는 이유는 사기 거래는 이상치로 간주하지 않기 위함.
    """
    normal = df[df["Class"] == 0][column]
    quantile_25 = np.percentile(normal.values, 25)
    quantile_75 = np.percentile(normal.values, 75)
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight

    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    outlier_idx = df[(df[column] < lowest_val) | (df[column] > highest_val) & (df["Class"] == 0)].index
    return outlier_idx

In [ ]:
# 모델 평가 함수

def get_clf_eval(y_test, pred, pred_proba=None, plot=False):
    """
    분류 성능 평가 지표를 출력하고, 필요시 PR/ROC 곡선 시각화.
    불균형 데이터에 적합한 지표들을 포함한다.
    """
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    pr_auc = average_precision_score(y_test, pred_proba)

    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}")
    print(f"F1 점수: {f1:.4f}, ROC AUC: {roc_auc:.4f}, PR AUC: {pr_auc:.4f}")


    if plot and pred_proba is not None:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        # PR 곡선
        precision_curve, recall_curve, _ = precision_recall_curve(y_test, pred_proba)
        ax1.plot(recall_curve, precision_curve, marker='.')
        ax1.set_title('Precision-Recall Curve')
        ax1.set_xlabel('Recall')
        ax1.set_ylabel('Precision')
        ax1.grid(True)
        # ROC 곡선
        fpr, tpr, _ = roc_curve(y_test, pred_proba)
        ax2.plot(fpr, tpr, marker='.')
        ax2.plot([0, 1], [0, 1], 'k--')
        ax2.set_title('ROC Curve')
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.grid(True)
        plt.tight_layout()
        plt.show()

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall,
            'f1': f1, 'roc_auc': roc_auc, 'pr_auc': pr_auc}

In [ ]:
# 임계값 최적화 함수

def find_optimal_threshold(y_test, pred_proba, metric='f1', plot=True): # plot 인자 추가 및 기본값 True
    """
    다양한 임계값을 테스트하여, 지정한 성능 지표가 최대가 되는 임계값을 찾는다.
    """
    thresholds = np.arange(0.1, 0.9, 0.05)
    scores = []

    for threshold in thresholds:
        pred = (pred_proba >= threshold).astype(int)
        if metric == 'f1':
            score = f1_score(y_test, pred)
        elif metric == 'precision':
            score = precision_score(y_test, pred)
        elif metric == 'recall':
            score = recall_score(y_test, pred)
        scores.append(score)

    best_idx = np.argmax(scores)
    best_threshold = thresholds[best_idx]
    best_score = scores[best_idx]
    print(f"최적 임계값: {best_threshold:.2f}, 최고 {metric} 점수: {best_score:.4f}")


    if plot: # plot이 True일 때만 시각화
        plt.figure(figsize=(8, 6))
        plt.plot(thresholds, scores, marker='o')
        plt.axvline(best_threshold, color='r', linestyle='--', label=f'최적 임계값: {best_threshold:.2f}')
        plt.title(f'임계값별 {metric} 점수')
        plt.xlabel('임계값')
        plt.ylabel(f'{metric} 점수')
        plt.grid(True)
        plt.legend()
        plt.show()

    return best_threshold

In [ ]:
# 모델 튜닝 함수

def tune_model_parameters(X_train, y_train, model_type, param_dist, n_iter=10, cv=3): # n_iter, cv 인자 추가
    """
    주어진 모델 타입과 하이퍼파라미터 범위로 최적 모델을 찾는다.
    """
    if model_type == 'rf':
        model = RandomForestClassifier(random_state=0)
    elif model_type == 'xgb':
        model = XGBClassifier(random_state=0)
    elif model_type == 'lgbm':
        model = LGBMClassifier(random_state=0)
    elif model_type == 'lr':
        model = LogisticRegression(random_state=0)

    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=n_iter, # n_iter 사용
        cv=cv, scoring='average_precision', n_jobs=-1, random_state=0 # cv 사용
    )
    random_search.fit(X_train, y_train)
    print(f"최적 하이퍼파라미터: {random_search.best_params_}")
    print(f"최고 평균 정밀도 점수: {random_search.best_score_:.4f}")
    return random_search.best_estimator_

In [ ]:
# 앙상블 모델 생성 함수

def create_ensemble_model(models):
    """
    여러 개의 개별 모델을 결합하여 소프트 투표 앙상블을 만든다.
    """
    estimators = [(f'model_{i}', model) for i, model in enumerate(models)]
    ensemble = VotingClassifier(estimators=estimators, voting='soft')
    return ensemble

In [ ]:
# 샘플링 방법 비교 함수

def find_best_sampling_method(X_train, y_train, classifier):
    """
    여러 오버샘플링 기법을 적용 후, 교차 검증 점수로 가장 성능이 좋은 방법을 선택한다.
    """
    samplings = {
        'smote': SMOTE(random_state=0),
        'borderline': BorderlineSMOTE(random_state=0),
        'adasyn': ADASYN(random_state=0)
    }
    best_score = 0
    best_method = None
    best_X, best_y = None, None

    print("샘플링 방법 비교 시작...")
    for name, sampler in samplings.items():
        print(f"\n{name} 샘플링 적용 중...")
        # 파이프라인을 사용하여 스케일링과 샘플링을 함께 수행
        pipeline = Pipeline([('sampler', sampler), ('classifier', classifier)])

        # 교차 검증 점수 계산
        # 불균형 데이터이므로 average_precision_score 사용
        scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='average_precision', n_jobs=-1)
        mean_score = scores.mean()
        print(f"{name} 샘플링 평균 교차 검증 점수 (Average Precision): {mean_score:.4f}")

        if mean_score > best_score:
            best_score = mean_score
            best_method = name
            # 최적 샘플링 방법으로 전체 데이터 리샘플링 (파이프라인 사용)
            best_sampler = sampler
            # best_X, best_y = best_sampler.fit_resample(X_train, y_train) # 이 부분은 main에서 수행

    print(f"\n최적 샘플링 방법: {best_method}, 점수: {best_score:.4f}")
    return best_method # 최적 방법 이름만 반환

In [ ]:
# 메인 실행 흐름

if __name__ == "__main__":
    # 데이터 로드
    train = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/train.csv")
    test = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/test.csv") # 오타 수정: credit_card -> credit-card

    # 데이터 탐색 (시간 단축을 위해 주석 처리 가능)
    # explore_data(train)

    # 데이터 분할 함수
    def get_train_test_dataset(df):
        X = df.drop('Class', axis=1)
        y = df['Class']
        return train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

    # 전처리 후 데이터 분할
    # 테스트 데이터도 동일한 전처리 함수 사용 (train_mode=False)
    train_processed = get_preprocessed_df(train, train_mode=True)
    test_processed = get_preprocessed_df(test, train_mode=False)

    X_train, X_test, y_train, y_test = get_train_test_dataset(train_processed)

    # 중요: 테스트 데이터에도 스케일링 적용을 위해 스케일러 객체 생성
    # 훈련 데이터로 fit 하고 테스트 데이터에는 transform만 적용
    scaler = StandardScaler()
    # 스케일링이 필요한 피처만 선택하여 적용 (V 특성들과 Amount_Scaled)
    # Hour 피처는 스케일링하지 않음
    scaled_features = [f'V{i}' for i in range(1, 29)] + ['Amount_Scaled']

    X_train[scaled_features] = scaler.fit_transform(X_train[scaled_features])
    X_test[scaled_features] = scaler.transform(X_test[scaled_features])
    test_processed[scaled_features] = scaler.transform(test_processed[scaled_features]) # 최종 예측에 사용될 테스트 데이터에도 적용

    print("\n데이터 스케일링 완료.")

    # 하이퍼파라미터 범위 정의 (시간 단축을 위해 범위 축소)
    lr_params = {'C': np.logspace(-3, 3, 10), 'penalty': ['l2'], 'solver': ['liblinear']} # l1은 liblinear와 saga만 지원, 범위 축소
    rf_params = {'n_estimators': [100, 200], 'max_depth': [10, 20, None], 'min_samples_split': [5, 10]} # 후보 값 줄임
    xgb_params = {'n_estimators': [100, 200], 'max_depth': [3, 5], 'learning_rate': [0.05, 0.1]} # 후보 값 줄임
    lgbm_params = {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1], 'num_leaves': [31, 63]} # 후보 값 줄임

    # 샘플링 방법 선택 및 적용
    # 샘플링 방법 비교는 기본 로지스틱 회귀로 수행
    base_classifier_for_sampling = LogisticRegression(random_state=0)
    best_sampling_method_name = find_best_sampling_method(X_train, y_train, base_classifier_for_sampling)

    # 최적 샘플링 방법으로 훈련 데이터 전체 리샘플링
    if best_sampling_method_name == 'smote':
        sampler = SMOTE(random_state=0)
    elif best_sampling_method_name == 'borderline':
        sampler = BorderlineSMOTE(random_state=0)
    elif best_sampling_method_name == 'adasyn':
        sampler = ADASYN(random_state=0)
    else:
        # 비교 결과가 없을 경우 기본 SMOTE 사용 또는 샘플링 하지 않음 선택
        sampler = SMOTE(random_state=0)
        print("최적 샘플링 방법 결정 실패 또는 비교 안 함. 기본 SMOTE 사용.")

    X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)
    print(f"\n최종 훈련 데이터에 {best_sampling_method_name} 샘플링 적용 완료. 형태: {X_train_resampled.shape}")

    # 각 모델 하이퍼파라미터 튜닝
    # 튜닝 시 사용하는 n_iter, cv 값 조정 (시간 단축)
    tuning_n_iter = 5 # 반복 횟수 감소
    tuning_cv = 2     # 교차 검증 폴드 수 감소

    print(f"\n모델 하이퍼파라미터 튜닝 시작 (RandomizedSearchCV n_iter={tuning_n_iter}, cv={tuning_cv})")

    # 파이프라인을 사용하여 샘플링과 모델 학습을 통합
    # 튜닝은 샘플링된 데이터로 직접 수행

    # Logistic Regression 튜닝 (파이프라인 제거)
    lr_model = tune_model_parameters(X_train_resampled, y_train_resampled, 'lr', {'C': np.logspace(-3, 3, 10)}, n_iter=tuning_n_iter, cv=tuning_cv)

    # Random Forest 튜닝 (파이프라인 제거)
    rf_model = tune_model_parameters(X_train_resampled, y_train_resampled, 'rf', rf_params, n_iter=tuning_n_iter, cv=tuning_cv)

    # XGBoost 튜닝 (파이프라인 제거)
    xgb_model = tune_model_parameters(X_train_resampled, y_train_resampled, 'xgb', xgb_params, n_iter=tuning_n_iter, cv=tuning_cv)

    # LightGBM 튜닝 (파이프라인 제거)
    lgbm_model = tune_model_parameters(X_train_resampled, y_train_resampled, 'lgbm', lgbm_params, n_iter=tuning_n_iter, cv=tuning_cv)

    print("\n모델 하이퍼파라미터 튜닝 완료.")

    # 모델 평가 및 최적 임계값 찾기
    models = [lr_model, rf_model, xgb_model, lgbm_model]
    model_names = ['Logistic Regression', 'Random Forest', 'XGBoost', 'LightGBM']
    model_results = {}

    print("\n개별 모델 성능 평가 시작")
    for name, model in zip(model_names, models):
        print(f"\n--- {name} 모델 평가 ---")
        # 테스트 데이터로 예측 확률 계산
        pred_proba = model.predict_proba(X_test)[:, 1]

        # 최적 임계값 찾기 (시각화 비활성화 - plot=False)
        opt_threshold = find_optimal_threshold(y_test, pred_proba, 'f1', plot=False)

        # 최적 임계값 적용하여 예측
        opt_pred = (pred_proba >= opt_threshold).astype(int)

        # 성능 평가 (시각화 비활성화 - plot=False)
        model_results[name] = get_clf_eval(y_test, opt_pred, pred_proba, plot=False)

    print("\n개별 모델 성능 평가 완료.")

    # 앙상블 모델 생성 및 학습
    print("\n앙상블 모델 학습 시작")
    # 앙상블 모델은 튜닝된 개별 모델들을 결합
    ensemble_model = create_ensemble_model([rf_model, xgb_model, lgbm_model]) # 로지스틱 회귀 제외 (선택 사항)
    # 앙상블 모델은 샘플링된 훈련 데이터로 학습
    ensemble_model.fit(X_train_resampled, y_train_resampled)
    print("앙상블 모델 학습 완료.")

    # 앙상블 모델 성능 평가
    print("\n--- 앙상블 모델 성능 평가 ---")
    ensemble_pred_proba = ensemble_model.predict_proba(X_test)[:, 1]

    # 앙상블 모델의 최적 임계값 찾기 (시각화 비활성화 - plot=False)
    ensemble_opt_threshold = find_optimal_threshold(y_test, ensemble_pred_proba, 'f1', plot=False)

    # 최적 임계값 적용하여 예측
    ensemble_opt_pred = (ensemble_pred_proba >= ensemble_opt_threshold).astype(int)

    # 성능 평가 (시각화 비활성화 - plot=False)
    model_results["Ensemble"] = get_clf_eval(y_test, ensemble_opt_pred, ensemble_pred_proba, plot=False)

    print("\n앙상블 모델 성능 평가 완료.")

    # 테스트 데이터 최종 예측
    print("\n테스트 데이터 최종 예측 시작")
    # 최종 앙상블 모델과 최적 임계값을 사용하여 예측
    final_pred_proba = ensemble_model.predict_proba(test_processed)[:, 1]
    final_pred = (final_pred_proba >= ensemble_opt_threshold).astype(int)
    print("테스트 데이터 최종 예측 완료.")

    # 제출 파일 생성
    print("\n제출 파일 생성 시작")
    submission = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/sample_submission.csv")
    submission["Class"] = final_pred
    submission.to_csv("./submission.csv", index=False)
    print("제출 파일 생성 완료: ./submission.csv")

제거된 이상치 데이터 수: 22173

데이터 스케일링 완료.
샘플링 방법 비교 시작...

smote 샘플링 적용 중...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

smote 샘플링 평균 교차 검증 점수 (Average Precision): 0.0081

borderline 샘플링 적용 중...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

borderline 샘플링 평균 교차 검증 점수 (Average Precision): 0.0139

adasyn 샘플링 적용 중...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

adasyn 샘플링 평균 교차 검증 점수 (Average Precision): 0.0076

최적 샘플링 방법: borderline, 점수: 0.0139

최종 훈련 데이터에 borderline 샘플링 적용 완료. 형태: (237876, 31)

모델 하이퍼파라미터 튜닝 시작 (RandomizedSearchCV n_iter=5, cv=2)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

최적 하이퍼파라미터: {'C': 0.004641588833612777}
최고 평균 정밀도 점수: 0.9990
최적 하이퍼파라미터: {'n_estimators': 200, 'min_samples_split': 10, 'max_depth': None}
최고 평균 정밀도 점수: 0.9999
최적 하이퍼파라미터: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1}
최고 평균 정밀도 점수: 1.0000
[LightGBM] [Info] Number of positive: 59469, number of negative: 59469
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.085091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7905
[LightGBM] [Info] Number of data points in the train set: 118938, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG